In [46]:
import time
import pickle
import requests
import random
from lxml.html import fromstring
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
import re
import pandas as pd
import os
from bs4 import BeautifulSoup
import copy
from itertools import cycle
import traceback

In [9]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())

Trying to download new driver from http://chromedriver.storage.googleapis.com/81.0.4044.138/chromedriver_win32.zip
Unpack archive C:\Users\blkyl\.wdm\drivers\chromedriver\81.0.4044.138\win32\chromedriver.zip


In [15]:
driver = webdriver.Chrome(r"C:\Users\blkyl\.wdm\drivers\chromedriver\81.0.4044.138\win32\chromedriver.exe")

In [16]:

driver.get('https://www.realtor.com/realestateandhomes-search/Los-Angeles_CA');
time.sleep(5) # Let the user actually see something!

#find the last page of realtor.com
soup = BeautifulSoup(driver.page_source, 'html.parser')
num_page = int(soup.find('div', class_="pagination-wrapper srp-pagination-bottom")
    .find_all('span')[-2]
    .text.strip())


la_house_info=[]
for i in range(num_page):
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    code_soup = soup.find_all('div', attrs={'class': 'photo-wrap'})
    for house in code_soup:
        results = house.find_all('a')
        for result in results:
            la_house_info.append(result['href'])
driver.quit()

In [17]:
la_house_info[:5]

['/realestateandhomes-detail/155-W-58th-St_Los-Angeles_CA_90037_M22496-25680',
 '/realestateandhomes-detail/1838-W-88th-Pl_Los-Angeles_CA_90047_M15937-65911',
 '/realestateandhomes-detail/14141-Dickens-St-Apt-305_Sherman-Oaks_CA_91423_M23888-95508',
 '/realestateandhomes-detail/746-S-Los-Angeles-St-Apt-604_Los-Angeles_CA_90014_M13749-42069',
 '/realestateandhomes-detail/1130-W-83rd-St_Los-Angeles_CA_90044_M15275-81597']

In [27]:
len(la_house_info)

11403

picked la house's urls 

In [18]:
with open('la_house_info.pickle', 'wb') as f:
    pickle.dump(la_house_info,f)

In [22]:
#window users needs to specify the path where webdriver can run!
#mac users doesn't need to specify it
driver = webdriver.Chrome(r"C:\Users\blkyl\.wdm\drivers\chromedriver\81.0.4044.138\win32\chromedriver.exe")

In [23]:
# Lrvine Ca house info

driver.get('https://www.realtor.com/realestateandhomes-search/Irvine_CA');
time.sleep(10)


#soup = BeautifulSoup(driver.page_source, 'html.parser')
#num_page = int(soup.find('div', class_="pagination-wrapper srp-pagination-bottom")
#    .find_all('span')[-2]
#    .text.strip())
#driver.quit()

lrvine_house_info=[]
for i in range(24):
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    code_soup = soup.find_all('div', attrs={'class': 'photo-wrap'})
    for house in code_soup:
        results = house.find_all('a')
        for result in results:
            lrvine_house_info.append(result['href'])
driver.quit()

In [24]:
lrvine_house_info[:5]

['/realestateandhomes-detail/138-Monroe-231_Irvine_CA_92620_M21962-94507',
 '/realestateandhomes-detail/125-Chantilly-101_Irvine_CA_92620_M21002-47822',
 '/realestateandhomes-detail/5200-Irvine-Blvd-Spc-181_Irvine_CA_92620_M11726-67911',
 '/realestateandhomes-detail/4-Petersburg_Irvine_CA_92620_M11247-41649',
 '/realestateandhomes-detail/1503-Terra-Bella_Irvine_CA_92602_M18219-00000']

picked lrvine's house urls

In [25]:
with open('lrvine_house_info.pickle', 'wb') as f:
    pickle.dump(lrvine_house_info,f)

THis function is to retrieve the house information using beautiful soup!!

In [28]:
sema = asyncio.BoundedSemaphore(30)
async def fetch(url, session):
    async with sema, session.get(url) as response:
        return await response.read()
    
    
async def run_urls(house_info):
    tasks = []
    async with ClientSession() as session:
        for url in house_info:
            house_url = 'https://www.realtor.com'+ url
            task = asyncio.ensure_future(fetch(house_url, session))
            tasks.append(task)
        responses = await asyncio.gather(*tasks)
        return responses


def house(soup):
    house_info = (soup.find('ul', class_= "property-meta list-horizontal list-style-disc list-spaced").find_all('span', attrs={'class':'data-value'}))
    house_info_dict ={}
    for i,v in enumerate(house_info):
        if i == 0:
            house_info_dict['beds'] = house_info[i].text
        if i == 1:
            house_info_dict['baths'] = house_info[i].text
        if i == 2:
            house_info_dict['sqft'] = house_info[i].text
        if i == 3:
            house_info_dict['lot'] = house_info[i].text
    
    return house_info_dict

def price(soup):
    return soup.find('span', attrs={'itemprop':'price'}).text.replace('\n','').strip() 

def pro_info(soup):
    result = soup.find_all('div', class_='key-fact-data ellipsis')
    pro_info_dict ={}
    for i,v in enumerate(result):
        if i == 0:
            pro_info_dict['status'] = result[i].text.replace('\n','').strip()
        if i == 1:
            pro_info_dict['price_per_sqft'] = result[i].text
        if i == 2:
            pro_info_dict['past_time'] = result[i].text
        if i == 3:
            pro_info_dict['house_type'] = result[i].text
        if i == 4:
            pro_info_dict['built_year'] = result[i].text
            
    return pro_info_dict 


def history_list(soup):
    history_tb = soup.find_all('table', class_= 'table')[1].find_all('tr')
    history_rows = [] 
    empty_row = {"Date": None, "Event": None, "Price": None ,"Price/Sq Ft": None, "Source": None}
    for row in history_tb[1:]:
        new_row = copy.copy(empty_row)
        columns = row.find_all("td")
        new_row['Date'] = columns[0].text.strip()
        new_row['Event'] = columns[1].text.strip()
        new_row['Price'] = columns[2].text.strip()
        new_row['Price/Sq Ft'] = columns[3].text.strip()
        new_row['Source'] = columns[4].text.strip()
        history_rows.append(new_row) 
    return history_rows


def school_list(soup):
    school_tb = soup.find_all('table', class_= 'table')[0].find_all('tr')
    school_rows = [] 
    empty_row = {"Rating": None, "School Name": None, "Grades": None ,"Distance": None}
    for row in school_tb[1:]:
        new_row = copy.copy(empty_row)
        columns = row.find_all("td")
        new_row['Rating'] = columns[0].text.strip()
        new_row['School Name'] = columns[1].text.strip()
        new_row['Grades'] = columns[2].text.strip()
        new_row['Distance'] = columns[3].text.strip()
        school_rows.append(new_row) 
    return school_rows

def neighbor_list(soup):
    neighbor_tb = soup.find_all('li',class_= 'clearfix text-center')
    neighbor_rows = []
    empty_row = {"price": None, "lat": None, "log": None}
    for row in neighbor_tb:
        new_row = copy.copy(empty_row)
        new_row['price'] = (row.find_all('div')[1]).text
        new_row['lat'] = (row.find_all('span')[-1]).find_all('meta')[0]["content"]
        new_row['log'] = (row.find_all('span')[-1]).find_all('meta')[1]["content"]
        neighbor_rows.append(new_row)  
    return neighbor_rows


In [34]:
from requests_html import HTMLSession
from pymongo import MongoClient
import pprint 
remote_client = MongoClient("127.0.0.1", 27017)
db = remote_client.house
la = db.la

retrieve data that I am interested in and then load it into mongoserver. 

In order to webscraping successfully from realtor.com which doesn't want to share their data with public, the following additional steps are needed:

1. rotate proxies (change IP address by each request)

2. time sleep

In [43]:
def get_proxies():
    url = '--' # I didn't put the website that sells ip address to rotate proxies
    response = requests.get(url)
    parser = fromstring(response.text)
    proxies = set()
    for i in parser.xpath('//tbody/tr')[:10]:
        if i.xpath('.//td[7][contains(text(),"yes")]'):
            proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
            proxies.add(proxy)
    return proxies

In [45]:
proxies = list(get_proxies())
proxies[:3]

['118.174.234.28:31994', '185.89.0.233:34927', '88.99.10.248:1080']

In [51]:
proxy=random.choice(proxies)

In [36]:
for i, url in enumerate(la_house_info):
    house_data = {}
    time.sleep(100)
    r = requests.get('https://www.realtor.com'+ url, proxies={'http':porxy, 'https':proxy})
    if r.status_code == 200:
        soup = BeautifulSoup(r.text, 'html.parser')
        house_data['id'] = i
        house_data['address'] = url
        house_data['price'] = price(soup)
        house_data['info'] = house(soup)
        house_data["area"] = (soup.find('div', class_='neighborhood-max-width-sm padding-bottom')
                                  .find('a').text)
        house_data['info'] = pro_info(soup)    
        house_data['history'] = history_list(soup)
        house_data['school'] = school_list(soup)
        house_data['neighbor'] = neighbor_list(soup)
        
    else:
        house_data['id'] = i
        house_data['url'] = url
        house_data['error'] = r.status_code

    la.insert_one(house_data)

In [39]:
house_data

{'id': 11402,
 'address': '/realestateandhomes-detail/1155-S-Grand-Ave-Apt-605_Los-Angeles_CA_90015_M25085-25663',
 'price': '$669,000',
 'info': {'status': 'Active',
  'price_per_sqft': '$637',
  'past_time': '79 days',
  'house_type': 'Condo/Townhome/Row Home/Co-Op',
  'built_year': '2008'},
 'area': 'Downtown Los Angeles',
 'history': [{'Date': '02/21/2020',
   'Event': 'Listed',
   'Price': '$669,000',
   'Price/Sq Ft': '$637',
   'Source': 'CLAW'},
  {'Date': '07/30/2009',
   'Event': 'Sold',
   'Price': '$415,000',
   'Price/Sq Ft': '$395',
   'Source': ''}],
 'school': [{'Rating': '4',
   'School Name': 'Ninth Street School',
   'Grades': 'K–5',
   'Distance': '1.0 mi'},
  {'Rating': '2',
   'School Name': 'John H. Liechty Middle School',
   'Grades': '6–8',
   'Distance': '1.0 mi'},
  {'Rating': '3',
   'School Name': 'Belmont Senior High School',
   'Grades': '9–12',
   'Distance': '1.4 mi'},
  {'Rating': 'NR',
   'School Name': 'Abram Friedman Occupational School',
   'Grades